# Double/Debiased Machine Learning

- Paper: [Chernozhukov, V., Chetverikov, D., Demirer, M., Duflo, E., Hansen, C., Newey, W., & Robins, J. (2018). Double/debiased machine learning for treatment and structural parameters.](https://academic.oup.com/ectj/article/21/1/C1/5056401?login=false)
- Python Package: [DoubleML](https://docs.doubleml.org/stable/index.html)

世の中の多くの現象は非線形な関係性が想定される。回帰分析は線形モデルであるため、モデルの定式化の誤りに起因するバイアスが生じかねない。

実際に関心のあるパラメータは少なく、交絡のコントロールのために入れている局外母数（nuisance parameters）は高次元になりがち。

局外母数を非線形モデルで表し、関心のあるパラメータは線形モデルで表現する部分線形モデル

$$
Y = \theta D + g(X) + e
$$

を作り、非線形モデル$g(X)$を機械学習で行いたい

## モーメント条件

### 外生性

単回帰モデル$Y = \alpha + \beta X + u$を例にとる。


:::{admonition} 外生性

説明変数$X$と誤差項$u$が

$$
E(u | X) = 0
$$

を満たすとき、$X$は外生変数であるという。

:::

また、外生性の条件は別の表現もできる

:::{admonition} $X$と$u$の直交
説明変数$X$が外生変数ならば、

$$
E(u) = 0, E(X, u) = 0
$$
:::

:::{dropdown} 証明
$$
E(X u) = E_{X}[ E(X u | X) ] = E_{X}[ X \underbrace{ E(u|X) }_{ =0 } ] = E_{X}(X\cdot 0) = 0
$$

- TODO: E(u) = 0の証明

上の式を直交条件と呼ぶ。
:::

さらに、共分散との関係も導出できる

:::{admonition} $X$と$u$の無相関
説明変数$X$が外生変数ならば、

$$
Cov(X, u) = 0
$$
:::

:::{dropdown} 証明

$$
Cov(X, u) = \underbrace{ E(X u) }_{ =0 }
- E(X) \underbrace{ E(u) }_{ =0 }
= 0
$$
:::


### OLS

単回帰モデル$Y = \alpha + \beta X + u$のOLS推定量$\beta$の確率極限は

$$
\DeclareMathOperator{\cov}{\text{Cov}}
\DeclareMathOperator{\var}{\text{Var}}
\DeclareMathOperator{\plim}{\text{plim}}
\plim \beta = \beta + \frac{\cov(X, u)}{\var(X)}
$$

となる。外生性が満たされるとき$\cov(X, u) = 0$であるため、$\plim \beta = \beta$となり、OLS推定量は母回帰係数の不偏推定量となる。

::::{dropdown} 証明
$$
\DeclareMathOperator{\cov}{\text{Cov}}
\DeclareMathOperator{\var}{\text{Var}}
\DeclareMathOperator{\plim}{\text{plim}}
\begin{align}
\plim \beta
&= \frac{\cov(X, Y)}{\var(X)}\\
&= \frac{\cov(X, \alpha + \beta X + u)}{\var(X)}\\
&= \frac{\cov(X, \alpha) + \cov(X, \beta X) + \cov(X, u) }{\var(X)}\\
&= \frac{\beta \var(X) + \cov(X, u) }{\var(X)}\\
&= \beta + \frac{\cov(X, u)}{\var(X)}
\end{align}
$$


:::{admonition}（参考）和の共分散
標本値、確率変数の和は、加える前の個々の共分散の和になる。すなわち、共分散においては分配法則が成り立つ。

$$
\cov(X + Z, Y) = \cov(X, Y) + \cov(Z, Y)
$$

参考：[確率統計 – 分散と共分散 – TauStation](http://taustation.com/statistics-variance-and-covariance/)
:::

:::{admonition} （参考）共分散と定数

$\alpha, \beta$は定数と考えると、

$$
\cov(X, \alpha)
= E(X \alpha) - E(X) E(\alpha)\\
= \alpha E(X) - \alpha E(X)\\
= 0
$$

であり

$$
\cov(X, \beta X)
= E(X \beta X) - E(X) E(\beta X)\\
= \beta E(X^2) - \beta E(X)^2\\
= \beta \var(X)
$$
:::
::::

### モーメント条件と非線形モデル

GMM推定量は非線形モデルに対しても用いることができる（末石2015, p.78）


## 部分線形モデル

結果変数$Y$、説明変数$X, Z$についての次のようなモデルを**部分線形モデル（Partially Linear Model）**という。

$$
\begin{align}
Y &= X^T \beta + g(Z) + u\\
E[u|X,Z] &= 0\\
E[u^2|X,Z] &= \sigma^2(X, Z)\\
\end{align}
$$

ここで、$g(\cdot)$は任意の関数（非線形の関数でもよい）である。$\sigma^2(\cdot, \cdot)$も未知の関数で、不均一分散を許容する。

関心のあるパラメータ$\beta$を解釈性の高い線形モデルで推定（パラメトリック推定）しつつ、影響を統制するためだけにモデルに投じている$Z$は関数形を特定せず推定（ノンパラメトリック推定）することができるため、部分線形モデルはセミパラメトリックモデルと呼ばれる。


### Robinson (1983)の推定量

$Z$で両辺の条件付き期待値をとった

$$
E[Y|Z] = E[X|Z]^T \beta + g(Z)
$$

を差し引くと

$$
Y - E[Y|Z] = (X - E[X|Z])^T \beta + u
$$

$\tilde{Y}_i = Y_i - E[Y_i|Z]$、$\tilde{X}_i = X_i - E[X_i|Z]$とおけば、OLS推定量の形になる

$$
\hat{\beta}_{\mathrm{inf}}=\left(\sum_{i=1}^n \tilde{X}_i \tilde{X}_i^{\prime}\right)^{-1} \sum_{i=1}^n \tilde{X}_i \tilde{Y}_i
$$

が$E[Y|Z], E[X|Z]$が未知のため実行不可能である。$E[Y|Z], E[X|Z]$をそれぞれのノンパラメトリック推定量で置換して推定を行うことは可能であり、その推定量はroot-N consistentで漸近正規性を持つ


### 参考

- [末石直也 - セミ・ノンパラメトリック計量分析 (京都大学)](https://sites.google.com/site/naoyasueishij/teaching/nonpara) 第5回 [部分線形モデルとセミパラメトリック推定量の性質](https://drive.google.com/file/d/0B6W_J4QoAI6wcWdwYkNwUU5DWTA/view?resourcekey=0--WtAUb3PgzgBpsw1XtvhzQ)
- 西山・人見（2023）『ノン・セミパラメトリック統計解析』、共立出版

### （疑問）Robinsonの推定量とDMLの違い、そうなった原因は？

#### Robinsonの推定量とDMLの違い

- 期待値を差し引くというFWL-style or Neiman-orthogonalityを使うのは一緒
- モーメント条件が議論の根幹なのでOLS推定量の形には限らない？？
- 差し引く期待値の推定をMLにしたのがDML

#### 新規性は

- ML推定ゆえoverfitting biasが入る → Cross Fitting
  - なんでMLだとbiasがはいる？



## Donsker条件

機械学習モデルの収束レートが遅い（Donsker条件）→Cross Fitting

[XユーザーのMasahiro Katoさん: 「Double/debiased machine learningはどういう手法かというと，機械学習はDonsker条件を満たさないので，サンプルを分割することによって，異なる部分集合間で独立だと思えるような局外母数の推定量を構築することで，収束率だけで漸近正規性を出す手法です．」 / X](https://twitter.com/masakat0/status/1314897112316870657)

## 考察

### 機械学習を使うと常にバイアスが入る？？

X-learnerとかどうなる？


## 参考

- 解説記事: [機械学習×計量経済学：Double/Debiased Machine Learning | Web日本評論](https://www.web-nippyo.jp/13331/)
- [[勉強会資料メモ] Double/Debiased ML - Speaker Deck](https://speakerdeck.com/masa_asa/debiased-ml?slide=32)

### DMLによるDID

[Double/debiased machine learning for difference-in-differences models | The Econometrics Journal | Oxford Academic](https://academic.oup.com/ectj/article/23/2/177/5722119)

- 解説: [DMLによる差分の差推定 - Speaker Deck](https://speakerdeck.com/masakat0/dmlniyoruchai-fen-falsechai-tui-ding)
- 関連: [Double-Debiased-Machine-learning-estimator-for-Difference-in-Difference-with-Multiple-Periods.pdf](https://www.researchgate.net/profile/Di-Liu-124/publication/370440876_DoubleDebiased_Machine-learning_estimator_for_Difference-in-Difference_with_Multiple_Periods/links/645015ce5762c95ac3676c6e/Double-Debiased-Machine-learning-estimator-for-Difference-in-Difference-with-Multiple-Periods.pdf)

### 講義動画（Youtube）

[Double Machine Learning for Causal and Treatment Effects - YouTube](https://www.youtube.com/watch?v=eHOjmyoPCFU&list=PLru50RuxzKFAsi9x3La3pidYURmi17ci6&index=4&t=479s)

- MLでのcausal parametersの推定は良いとは限らない
- double or orthogonalized MLとsample splittingによって、causal parametersの高精度な推定が可能

Partially Linear Modelを使う

$$
Y = D \theta_0 + g_0(Z) + U
, \hspace{1em} E[U|Z, D] = 0
$$

- MLをそのまま使うと一致推定量にならない（例えば$Y - D$で$g_0(Z)$をRandom Forestで学習しても、予測精度は良いがバイアスがある）
- FWL定理を用いて、残差の回帰にするとよい

$$
\hat{W} = Y - \hat{E[Y|Z]}\\
\hat{V} = D - \hat{E[D|Z]}
$$


モーメント条件

1. Regression adjustment: $E[(Y - D \theta_0 - g_0(Z) ) D] = 0$
2. "propensity score adjustment": $E[(Y - D \theta_0) (D - E[D|Z])] = 0$
3. Neyman-orthogonal (semi-parametrically efficient under homoscedasticity): $E[(\hat{W} - \hat{V}\theta_0) \hat{V}] = E[\{(Y - E[Y|Z]) - (D - E[D|Z])\theta_0\} (D - E[D|Z])] = 0$

3は不偏


Sample Splitting


Splittingによるefficiencyの低下問題

- 2個に分けて2回やって平均とればfull efficiency → k個に分けての分析をk回やって平均とってもfull efficiency



## 応用研究

[[2002.08536] Debiased Off-Policy Evaluation for Recommendation Systems](https://arxiv.org/abs/2002.08536)
- PR: [AI Lab、推薦システム分野におけるトップカンファレンス「RecSys2021」にて共著論文採択 ー高次元なデータを使った意思決定評価の信頼性を改善ー | 株式会社サイバーエージェント](https://www.cyberagent.co.jp/news/detail/id=26585)

## 関連研究

[[2305.04174] Root-n consistent semiparametric learning with high-dimensional nuisance functions under minimal sparsity](https://arxiv.org/abs/2305.04174)

- 先行研究まとめがある

[Library Flow Chart — econml 0.15.0 documentation](https://econml.azurewebsites.net/spec/flowchart.html)
- 派生モデルの使い分けについて